<a href="https://colab.research.google.com/github/hashmil/MidiGen/blob/main/MidiGen_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Initial Setup
run_on_colab = True

if(run_on_colab):
    from google.colab import drive
    drive.mount('/content/drive')

In [11]:
# Setup and Load Model and n_vocab
import pickle
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from music21 import note, chord, stream, instrument
from tqdm import tqdm


with open('/content/drive/MyDrive/MIDI/vivaldi/pickle/notes.pkl', 'rb') as filepath:
    notes = pickle.load(filepath)
with open('/content/drive/MyDrive/MIDI/vivaldi/pickle/n_vocab.pkl', 'rb') as filepath:
    n_vocab = pickle.load(filepath)

In [12]:
def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

In [13]:
# Load the model structure
sequence_length = 100
network_input = numpy.reshape(numpy.array([0]*sequence_length), (1, sequence_length, 1))
model = create_network(network_input, n_vocab)

In [14]:
# Load the model weights
weights_path = '/content/drive/MyDrive/MIDI/vivaldi/weights/weights-99-0.0860.h5'  # Replace with your model weights path
model.load_weights(weights_path)

In [ ]:
# Load processed notes
with open('/content/drive/MyDrive/MIDI/vivaldi/pickle/notes.pkl', 'rb') as filepath:
    notes = pickle.load(filepath)

# Prepare the dictionary to convert notes to integers
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
n_vocab = len(set(notes))

# Prepare the network input
sequence_length = 100
network_input = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])

# Reshape the input and normalize
n_patterns = len(network_input)
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)

# Generate a random starting point for prediction
start = numpy.random.randint(0, len(network_input)-1)
pattern = network_input[start]

# Generate Music
prediction_output = []
for note_index in tqdm(range(500), desc="Generating music"):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern = numpy.append(pattern, index)
    pattern = pattern[1:len(pattern)]

In [ ]:
# Convert Output to MIDI
offset = 0
output_notes = []

for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/content/drive/MyDrive/MIDI/vivaldi/output/output-002.mid')